In [ ]:
!ls /dev/tty*

In [ ]:
import os
import sys
import time
import smbus
import math
import numpy as np

from imusensor.MPU9250 import MPU9250

from jetbot import Robot
import PIL

import serial


robot = Robot()


# GPS (UART)

ser = serial.Serial("/dev/ttyACM0") # Address of GPS Device connected via USB
gpgga_info = "$GPGGA," # Extract GPGGA line from GPS
GPGGA_buffer = 0
NMEA_buff = 0

# IMU Address
address = 0x68
bus = smbus.SMBus(0)
imu = MPU9250.MPU9250(bus, address)
imu.begin()
imu.loadCalibDataFromFile("docs/calib_real4.json")


def convert_to_degrees(raw_value):
    decimal_value = raw_value/100.00
    degrees = int(decimal_value)
    mm_mmmm = (decimal_value - int(decimal_value))/0.6
    position = degrees + mm_mmmm
#     position = "%.4f" %(position)
    return position


# Magneto Meter

def currentHeading(): #Get current Headding/Facing direction from Magnetometer (0 to 360 deg)
    
    imu.readSensor()
    imu.computeOrientation()
    
        
    magX = imu.MagVals[0] 
    magY = imu.MagVals[1]
    M_PI = math.pi
    offSet = 30 # Correction Angle
    
    if (magY > 0):
        heading = math.atan(magX/magY)*(180/M_PI) - 90
        
    elif (magY < 0):
        heading = math.atan(magX/magY)*(180/M_PI) - 270
        
    elif (magY == 0 and magX < 0):
        heading = 180
        
    elif (magY == 0 and magX > 0): 
        heading = 0
        
    heading = heading + offSet
    if heading > 360:
        heading = heading - 360;
    elif heading < 0:
        heading = heading + 360;
        
    return heading

# Navigation
def getSteering(currHead, currLat, currLon, targLat, targLon): # Steering angle towards target locaation wrt target location
    # currHead must be from -360 to 360;
    x = math.cos(np.deg2rad(targLat)) * math.sin(np.deg2rad(currLon-targLon));
    y = math.cos(np.deg2rad(currLat)) * math.sin(np.deg2rad(targLat)) - math.sin(np.deg2rad(currLat))* math.cos(np.deg2rad(targLat)) * math.cos(np.deg2rad(currLon-targLon));
    mag = currHead;
    steering = -np.rad2deg(math.atan2(x,y)) - mag;
    if steering >= 180:
        steering = steering - 360;
    elif steering <= -180:
        steering = steering + 360;
    return steering
        
def getDistance(curLat, curLon, lat2, lon2):  # generally used geo measurement function
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - curLat * math.pi / 180
    dLon = lon2 * math.pi / 180 - curLon * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(curLat * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c;
    return d * 1000; # meters

def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0):
        L = leftDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
        
    if (steeringAngle < -90) and (steeringAngle >= -180):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed
      
    
    if (steeringAngle > 0) and steeringAngle <= 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
    
    if (steeringAngle > 90) and steeringAngle > 90:
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down
        

    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity

# robot.stop

try:
    while True:
        imu.readSensor() # Read IMU Sensor
        imu.computeOrientation()
        
        received_data = (str)(ser.readline()) #read NMEA string received
        GPGGA_data_available = received_data.find(gpgga_info)   #check for NMEA GPGGA string
        
        if (GPGGA_data_available>0): # when GPS is active
            GPGGA_buffer = received_data.split("$GPGGA,",1)[1]  #store data coming after “$GPGGA,” string
            NMEA_buff = (GPGGA_buffer.split(','))
            nmea_time = []
            nmea_latitude = []
            nmea_longitude = []
            nmea_time = float(NMEA_buff[0]) + 53000                    #extract time from GPGGA string
            nmea_latitude = NMEA_buff[1]                #extract latitude from GPGGA string
            nmea_latitude_dir = NMEA_buff[2]                #Lat direction
            nmea_longitude = NMEA_buff[3]               #extract longitude from GPGGA string
            nmea_long_dir = NMEA_buff[4]                #Long direction  

            GPS_Fix = NMEA_buff[5]
            satellite_count = NMEA_buff[6]
            Alt = NMEA_buff[8]

            lat = float(nmea_latitude)
            lat = convert_to_degrees(lat)
            longi = float(nmea_longitude)
            longi = convert_to_degrees(longi)
        
            currentHeading1 = currentHeading()
            print("currentHeading:",currentHeading1)
            currentLat = lat
            currentLon = longi
            
# #             targetLat = 17.747145
#             targetLat = 17.747200833333334
# #             targetLon = 83.2378320
#             targetLon = 83.23780333333335
    
            targetLat = 17.7472805
            targetLon =83.23778333333333
    
#     17.747200833333334 , 83.23780333333335
    
            print('Time: ',nmea_time)
            print('Current Location: ',currentLat,',',currentLon)
            

            driveSpeed = 0.2

            steerAng = getSteering(currentHeading1, currentLat, currentLon, targetLat, targetLon)
            distFromTraget = getDistance(currentLat, currentLon, targetLat, targetLon)
            
            print('Steering Angle: ',steerAng)
            print('Distance from Target: ',distFromTraget)

            if distFromTraget > 3:
                drive = differentialDrive(steerAng, driveSpeed)
                LWSpeed = drive[0]
                RWSpeed = drive[1]
                print('Speed:\t',LWSpeed,'\t,',RWSpeed)
                robot.set_motors(LWSpeed,RWSpeed)

            else:
                print("Goal Reached")
                robot.set_motors(0,0)
                break     
        
except KeyboardInterrupt:
    robot.stop()
    sys.exit(0)



#  robot.set_motors(0.5,0.5)

In [ ]:
robot.stop()

In [ ]:
def mapf(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def differentialDrive(steeringAngle, driveSpeed):
    # steeringAngle must be from -180 to 180.

    leftDriveSpeed = driveSpeed;
    rightDriveSpeed = driveSpeed;
    if (steeringAngle <= 0):
        L = leftDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow left down
        R = rightDriveSpeed

    
    if (steeringAngle > 0):
        L = leftDriveSpeed
        R = rightDriveSpeed - 2*mapf(abs(steeringAngle), 0,180, 0,driveSpeed) # slow right down

    L_R_velocity = [L, R,mapf(abs(steeringAngle), 0,180, 0,driveSpeed)]
    return L_R_velocity

differentialDrive(-40,2)
